## Encode Test Images with CLIP for Demo

For Image Retrieval given text and image prompts demo. Needs image vectors pre-extracted for corpus of test images (around 1000).

In [1]:
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
import jax
import jax.numpy as jnp
import json
import matplotlib.pyplot as plt
import numpy as np
import requests
import os

from PIL import Image
from transformers import CLIPProcessor, FlaxCLIPModel

Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003834.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003834.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003834.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003834.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003834.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003834.106129': Permission denied
Could not open any log file.
Could not open the log

In [3]:
jax.device_count()

INFO:absl:Starting the local TPU driver.
INFO:absl:Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: local://
INFO:absl:Unable to initialize backend 'gpu': Not found: Could not find registered platform with name: "cuda". Available platform names are: TPU Interpreter Host
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003835.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003835.106129': Permission denied
Could not open any log file.


8

Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003835.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003835.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003835.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003835.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003835.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003835.106129': Permission denied
Could not open any log file.
Could not open the log

In [4]:
DATA_DIR = "/home/shared/data"

IMAGES_DIR = os.path.join(DATA_DIR, "RSICD_images")
CAPTIONS_FILE = os.path.join(DATA_DIR, "dataset_rsicd.json")

VECTOR_FILE = os.path.join("image-vectors.tsv")

### Test Images

In [5]:
image2captions = {}
with open(CAPTIONS_FILE, "r") as fcap:
    data = json.loads(fcap.read())
for image in data["images"]:
    if image["split"] == "test":
        filename = image["filename"]
        sentences = []
        for sentence in image["sentences"]:
            sentences.append(sentence["raw"])
        image2captions[filename] = sentences
    
len(image2captions)

1093

### Encode Image using Model

In [6]:
model = FlaxCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003839.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003839.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003839.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003839.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003840.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003840.106129': Permission denied
Could not open any log file.
Could not open the log

In [7]:
def encode_image(image_file, model, processor):
    image = Image.fromarray(plt.imread(os.path.join(IMAGES_DIR, image_file)))
    inputs = processor(images=image, return_tensors="jax")
    image_vec = model.get_image_features(**inputs)
    return np.array(image_vec).reshape(-1)

image_filename = list(image2captions.keys())[0]
image_vec = encode_image(image_filename, model, processor)
image_vec.shape, type(image_vec)

Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003850.106129': Permission denied
Could not open any log file.


((512,), numpy.ndarray)

### Generate Image Encodings for all Test Images

In [8]:
num_written = 0
fvec = open(VECTOR_FILE, "w")
for image_file in image2captions.keys():
    if num_written % 100 == 0:
        print("{:d} images processed".format(num_written))
    image_vec = encode_image(image_file, model, processor)
    image_vec_s = ",".join(["{:.7e}".format(x) for x in image_vec])
    fvec.write("{:s}\t{:s}\n".format(image_file, image_vec_s))
    num_written += 1
    
print("{:d} images processed, COMPLETE".format(num_written))
fvec.close()

0 images processed


Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003850.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003850.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003850.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003850.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003850.106129': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-bcec0017-w-0.palsujit.log.INFO.20210711-003850.106129': Permission denied
Could not open any log file.
Could not open the log

100 images processed
200 images processed
300 images processed
400 images processed
500 images processed
600 images processed
700 images processed
800 images processed
900 images processed
1000 images processed
1093 images processed, COMPLETE
